In [1]:
#include <iostream>
#include <algorithm>
#include <numeric>
#include <vector>

In [2]:
int arr[3] {1, 2, 3};

In [3]:
std::accumulate(arr, arr+3, 1, std::multiplies<int>())

6

In [8]:
std::vector<int> v{1, 2, 3, 4, 5, 6, 7, 8, 9, 10};

In [13]:
std::accumulate(arr, arr+3, 1, std::multiplies<int>())

0

In [11]:
std::reduce(v.begin(), v.end(), 1, std::multiplies<int>())

input_line_20:2:7: error: no member named 'reduce' in namespace 'std'
 std::reduce(v.begin(), v.end(), 1, std::multiplies<int>())
 ~~~~~^


Interpreter Error: 

In [6]:
std::accumulate(arr, 3, std::multiplies<int>())

input_line_12:2:2: error: no matching function for call to 'accumulate'
 std::accumulate(arr, 3, std::multiplies<int>())
 ^~~~~~~~~~~~~~~
/home/olszewskip/miniconda3/envs/xeus_cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/bits/stl_numeric.h:120:5: note: candidate template ignored: deduced conflicting types for parameter
      '_InputIterator' ('int *' vs. 'int')
    accumulate(_InputIterator __first, _InputIterator __last, _Tp __init)
    ^
/home/olszewskip/miniconda3/envs/xeus_cling/bin/../lib/gcc/x86_64-conda_cos6-linux-gnu/7.3.0/../../../../x86_64-conda_cos6-linux-gnu/include/c++/7.3.0/bits/stl_numeric.h:146:5: note: candidate function template not viable: requires 4 arguments, but 3 were
      provided
    accumulate(_InputIterator __first, _InputIterator __last, _Tp __init,
    ^


Interpreter Error: 

In [3]:
struct Matrix {
    double* data = NULL;
    ~Matrix() {
        delete[] data;
    }
}

In [35]:
for (auto& it : arr_)
    std::cout << it << std::endl;

0
1
2


In [33]:
std::copy_n(arr_, 3, arr)

@0x7ffd0ebb4f48

In [29]:
*arr_

0

In [5]:
*arr

1

In [7]:
int* ptr = arr;

In [9]:
*ptr

1

In [14]:
template<unsigned int dim>
struct test {
    unsigned int shape[dim];
    test(usigned int shape_) {
        std::copy_n(shape_, dim, shape)
    }
};

In [ ]:
template<unsigned int dim_>
struct MatrixD : Matrix {
    const unsigned int dim = dim_;
    unsigned int shape[dim_];
    MatrixD(unsigned int shape_[]) {
        std::copy_n(shape_, dim, shape);
        data = new T[std::accumulate(shape_, dim_, std::multiplies<int>())];
    }
    unsigned_int* elem_ptr(unsigned int index[dim]) {}
    
}

---

In [5]:
#include<valarray>

In [7]:
std::valarray<int> varr = { 10, 2, 20, 1, 30 };

In [9]:
varr = varr * std::log(varr);

In [11]:
varr[0]

20